<a href="https://colab.research.google.com/github/jmarrietar/ocular/blob/master/notebooks/PAWS_TPU_DR_PyTorch_XLA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
"""
To Do: 
  - Dont Donwload several Times ResNet model. How?. 
"""

'\nTo Do: \n  - Dont Donwload several Times ResNet model. How?. \n'

In [1]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8.1-cp37-cp37m-linux_x86_64.whl

     |████████████████████████████████| 145.0 MB 10 kB/s 
     |████████████████████████████████| 57 kB 2.8 MB/s 
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.278 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatible.


In [2]:
!pip uninstall torch -y
!pip install torch==1.8.1

Found existing installation: torch 1.9.0+cu102
Uninstalling torch-1.9.0+cu102:
  Successfully uninstalled torch-1.9.0+cu102
     |████████████████████████████████| 804.1 MB 2.2 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.8.1 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.8.1 which is incompatible.


In [3]:
import numpy as np
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.utils.utils as xu
import torchvision
from torchvision import datasets, transforms
import argparse
import yaml
import pprint
import logging
import os

In [4]:
pwd

'/content'

In [5]:
pip install -U PyYAML

     |████████████████████████████████| 636 kB 5.2 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
pwd

'/content'

In [7]:
!git clone -b feature/DR-images-v2 https://github.com/jmarrietar/suncet.git

Cloning into 'suncet'...
remote: Enumerating objects: 356, done.
remote: Counting objects: 100% (356/356), done.
remote: Compressing objects: 100% (235/235), done.
remote: Total 356 (delta 215), reused 248 (delta 118), pack-reused 0
Receiving objects: 100% (356/356), 1.11 MiB | 8.36 MiB/s, done.
Resolving deltas: 100% (215/215), done.


In [8]:
cd suncet

/content/suncet


In [9]:
!mkdir datasets
!mkdir datasets/dr
!mkdir logs

In [10]:
!python download.py -d sample@2000

Downloading...
From: https://drive.google.com/uc?id=1PB7uGd-dUnZKnKZpZl-HvE1DVcWgX50F
To: /content/suncet/datasets/dr/sample@2000.zip
214MB [00:00, 274MB/s]


In [11]:
parser = argparse.ArgumentParser()
parser.add_argument(
    '--fname', type=str,
    help='name of config file to load',
    default='configs.yaml')
parser.add_argument(
    '--devices', type=str, nargs='+', default=['cuda:0'],
    help='which devices to use on local machine')
parser.add_argument(
    '--sel', type=str,
    help='which script to run',
    choices=[
        'paws_train',
        'suncet_train',
        'fine_tune',
        'snn_fine_tune'
    ])

_StoreAction(option_strings=['--sel'], dest='sel', nargs=None, const=None, default=None, type=<class 'str'>, choices=['paws_train', 'suncet_train', 'fine_tune', 'snn_fine_tune'], help='which script to run', metavar=None)

In [12]:
args = parser.parse_args(['--sel', 'paws_train',
                            '--fname', 'configs/paws/dr_train.yaml'
])

In [13]:
fname = args.fname
sel = args.sel

In [14]:
logging.basicConfig()
logger = logging.getLogger()

logger.info(f'called-params {sel} {fname}')

In [15]:
# -- load script params
params = None
with open(fname, 'r') as y_file:
    #params = yaml.load(y_file, Loader=yaml.FullLoader)
    params = yaml.load(y_file)
    logger.info('loaded params...')
    #if rank == 0:
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(params)

{   'criterion': {   'classes_per_batch': 2,
                     'me_max': True,
                     'sharpen': 0.25,
                     'supervised_imgs_per_class': 8,
                     'supervised_views': 1,
                     'temperature': 0.1,
                     'unsupervised_batch_size': 32},
    'data': {   'color_jitter_strength': 1.0,
                'data_seed': None,
                'dataset': 'dr',
                'label_smoothing': 0.1,
                'multicrop': 6,
                'normalize': True,
                'root_path': 'datasets/',
                's_image_folder': 'dr/sample@2000/',
                'subset_path': 'dr_subsets',
                'u_image_folder': 'dr/sample@2000/',
                'unique_classes_per_rank': False,
                'unlabeled_frac': 0.9},
    'logging': {'folder': 'logs/', 'write_tag': 'paws'},
    'meta': {   'copy_data': True,
                'device': 'cuda:0',
                'load_checkpoint': False,
               

In [16]:
args = params

In [17]:
# Define Parameters
FLAGS = {}
FLAGS['data_dir'] = "/tmp/cifar"
FLAGS['batch_size'] = 32
FLAGS['num_workers'] = 2
FLAGS['learning_rate'] = 0.02
FLAGS['momentum'] = 0.9
FLAGS['num_epochs'] = 10
FLAGS['num_cores'] = 8 if os.environ.get('TPU_NAME', None) else 1
FLAGS['log_steps'] = 20
FLAGS['metrics_debug'] = False
FLAGS['model_name'] = args["meta"]["model_name"]

In [18]:
model_name = args["meta"]["model_name"]
output_dim = args["meta"]["output_dim"]
multicrop = args["data"]["multicrop"]

# -- CRITERTION
reg = args["criterion"]["me_max"]
supervised_views = args["criterion"]["supervised_views"]
classes_per_batch = args["criterion"]["classes_per_batch"]
s_batch_size = args["criterion"]["supervised_imgs_per_class"]
u_batch_size = args["criterion"]["unsupervised_batch_size"]
temperature = args["criterion"]["temperature"]
sharpen = args["criterion"]["sharpen"]

# -- DATA
unlabeled_frac = args["data"]["unlabeled_frac"]
color_jitter = args["data"]["color_jitter_strength"]
normalize = args["data"]["normalize"]
root_path = args["data"]["root_path"]
s_image_folder = args["data"]["s_image_folder"]
u_image_folder = args["data"]["u_image_folder"]
dataset_name = args["data"]["dataset"]
subset_path = args["data"]["subset_path"]
unique_classes = args["data"]["unique_classes_per_rank"]
label_smoothing = args["data"]["label_smoothing"]
data_seed = None

copy_data = args["meta"]["copy_data"]
use_pred_head = args["meta"]["use_pred_head"]

use_fp16 = args["meta"]["use_fp16"]

# -- OPTIMIZATION
wd = float(args["optimization"]["weight_decay"])
num_epochs = args["optimization"]["epochs"]
warmup = args["optimization"]["warmup"]
start_lr = args["optimization"]["start_lr"]
lr = args["optimization"]["lr"]
final_lr = args["optimization"]["final_lr"]
mom = args["optimization"]["momentum"]
nesterov = args["optimization"]["nesterov"]


# -- META
load_model = args["meta"]["load_checkpoint"]
r_file = args["meta"]["read_checkpoint"]


# -- LOGGING
folder = args["logging"]["folder"]
tag = args["logging"]["write_tag"]
# ----------------------------------------------------------------------- #

In [19]:
crop_scale = (0.14, 1.0) if multicrop > 0 else (0.08, 1.0)
mc_scale = (0.05, 0.14)
mc_size = 96

In [20]:
import logging
import sys
from collections import OrderedDict
import traceback
import numpy as np

import torch

import src.resnet as resnet
import src.wide_resnet as wide_resnet
from src.utils import (
    gpu_timer,
    init_distributed,
    WarmupCosineSchedule,
    CSVLogger,
    AverageMeter,
)
from src.losses import init_paws_loss, make_labels_matrix
from src.data_manager import init_data, make_transforms, make_multicrop_transform
from src.sgd import SGD
from src.lars import LARS

import torchvision.models as models
from torch.hub import load_state_dict_from_url

#import apex
from torch.nn.parallel import DistributedDataParallel

In [23]:
from src.paws_train import init_model
from src.data_manager import ImageNet
from src.data_manager import GaussianBlur

## PyTorch/XLA ResNet18/CIFAR10 (GPU or TPU)

In [27]:
import argparse
import logging
import os
import time
import gdown
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.utils.serialization as xser
import torch_xla.utils.utils as xu
import torchvision
from torchvision import datasets, transforms
from src.utils import (
    init_distributed,
    WarmupCosineSchedule,
    CSVLogger,
    AverageMeter,
)
from src.utils import (
    AllGather,
    AllReduce
)

In [28]:
def download(data, url):
    # Download dataset
    import zipfile
    url = url
    output = "{}.zip".format(data)
    gdown.download(url, output, quiet=False)

    # Uncompress dataset
    local_zip = '{}.zip'.format(data)
    zip_ref = zipfile.ZipFile(local_zip, "r")
    zip_ref.extractall()
    zip_ref.close()

In [29]:
data_samples = {
    "sample@200": "https://drive.google.com/uc?id=1FfV7YyDJvNUCDP5r3-8iQfZ2-xJp_pgb",
    "sample@500": "https://drive.google.com/uc?id=1dHwUqpmSogEdjAB9rwDUL-OKFRUcVXte",
    "sample@1000": "https://drive.google.com/uc?id=1DPZrHrj3Bdte5Dc6NCZ33CAqMG-Oipa2",
    "sample@2000": "https://drive.google.com/uc?id=1PB7uGd-dUnZKnKZpZl-HvE1DVcWgX50F",
    "sample@3000": "https://drive.google.com/uc?id=1_yre5K9YYvJgSrT4xvrI8eD_htucIywA",
    "sample@4000_images": "https://drive.google.com/uc?id=1dqVB8EozEpwWzyuU80AauoQmsiw3Gtm2",
    "sample@20000": "https://drive.google.com/uc?id=1MTDpLzpmhSiZq2jSdmHx2UDPn9FC8gzO",
    "val-voets-tf": "https://drive.google.com/uc?id=1VzVgMGTkBBPG2qbzLunD9HvLzH6tcyrv",
    "train_voets": "https://drive.google.com/uc?id=1AmcFh1MOOZ6aqKm2eO7XEdgmIEqHKTZ5",
    "voets_test_images": "https://drive.google.com/uc?id=15S_V3B_Z3BOjCT3AbO2c887FyS5B0Lyd"
}

In [30]:
UNLABELED = 'sample@2000'

In [31]:
URL_UNLABELED = data_samples[UNLABELED]
download(UNLABELED, URL_UNLABELED)

Downloading...
From: https://drive.google.com/uc?id=1PB7uGd-dUnZKnKZpZl-HvE1DVcWgX50F
To: /content/suncet/sample@2000.zip
214MB [00:00, 341MB/s]


Only run the below commented cell if you would like a nightly release

### Define Parameters



In [37]:
multicrop=multicrop
tau=temperature
T=sharpen
me_max=reg


"""
Make semi-supervised PAWS loss

:param multicrop: number of small multi-crop views
:param tau: cosine similarity temperature
:param T: target sharpenning temperature
:param me_max: whether to perform me-max regularization
"""

def sharpen_func(p):
    sharp_p = p**(1./T)
    sharp_p /= torch.sum(sharp_p, dim=1, keepdim=True)
    return sharp_p

def snn(query, supports, labels, tau):

    softmax = torch.nn.Softmax(dim=1)
    """ Soft Nearest Neighbours similarity classifier """
    # Step 1: normalize embeddings
    query = torch.nn.functional.normalize(query)
    supports = torch.nn.functional.normalize(supports)

    # Step 2: gather embeddings from all workers
    supports = AllGather.apply(supports)

    # Step 3: compute similarlity between local embeddings

    #result = softmax(query @ supports.T / tau) @ labels
    result = torch.matmul(softmax(torch.matmul(query, supports.T / tau)), labels)

    return result

def my_loss_func(
    anchor_views,
    anchor_supports,
    anchor_support_labels,
    target_views,
    target_supports,
    target_support_labels,
    sharpen=sharpen_func,
    snn=snn
):
    # -- NOTE: num views of each unlabeled instance = 2+multicrop
    batch_size = len(anchor_views) // (2+multicrop)

    # Step 1: compute anchor predictions
    probs = snn(anchor_views, anchor_supports, anchor_support_labels, tau)

    # Step 2: compute targets for anchor predictions
    with torch.no_grad():
        targets = snn(target_views, target_supports, target_support_labels, tau)
        targets = sharpen(targets)
        if multicrop > 0:
            mc_target = 0.5*(targets[:batch_size]+targets[batch_size:])
            targets = torch.cat([targets, *[mc_target for _ in range(multicrop)]], dim=0)
        targets[targets < 1e-4] *= 0  # numerical stability

    # Step 3: compute cross-entropy loss H(targets, queries)
    #loss = torch.mean(torch.sum(torch.log(probs**(-targets)), dim=1))

    criterion = torch.nn.CrossEntropyLoss()
    targets2 = targets.argmax(-1)
    loss = criterion(probs, targets2)

    # Step 4: compute me-max regularizer
    rloss = 0.
    if me_max:
        avg_probs = AllReduce.apply(torch.mean(sharpen(probs), dim=0))
        rloss -= torch.sum(torch.log(avg_probs**(-avg_probs)))

    return loss, rloss

----

----

In [53]:
SERIAL_EXEC = xmp.MpSerialExecutor()
# Only instantiate model weights once in memory.
#WRAPPED_MODEL = xmp.MpModelWrapper(ResNet18())

def train_resnet18():
  torch.manual_seed(1)

  ############# PAWS CODE ##################

  device = xm.xla_device()

  # -- init model
  encoder = init_model(
    device=device,
    model_name=model_name,
    use_pred=use_pred_head,
    output_dim=output_dim,
  )

  # -- init losses
  #paws = init_paws_loss(multicrop=multicrop, tau=temperature, T=sharpen, me_max=reg)
  # -- assume support images are sampled with ClassStratifiedSampler

  # TO DO: Hacer el smooth de los labels Matrix.

# -- make data transforms
  transform, init_transform = make_transforms(
    dataset_name=dataset_name,
    subset_path=subset_path,
    unlabeled_frac=unlabeled_frac,
    training=True,
    split_seed=data_seed,
    crop_scale=crop_scale,
    basic_augmentations=False,
    color_jitter=color_jitter,
    normalize=normalize,
  )
  multicrop_transform = (multicrop, None)
  if multicrop > 0:
    multicrop_transform = make_multicrop_transform(
        dataset_name=dataset_name,
        num_crops=multicrop,
        size=mc_size,
        crop_scale=mc_scale,
        normalize=normalize,
        color_distortion=color_jitter,
    )

# -- init data-loaders/samplers
  (
    unsupervised_loader,
    unsupervised_sampler,
    supervised_loader,
    supervised_sampler,
) = init_data(
    dataset_name=dataset_name,
    transform=transform,
    init_transform=init_transform,
    supervised_views=supervised_views,
    u_batch_size=u_batch_size,
    s_batch_size=s_batch_size,
    unique_classes=unique_classes,
    classes_per_batch=classes_per_batch,
    multicrop_transform=multicrop_transform,
    world_size=xm.xrt_world_size(),
    rank=xm.get_ordinal(),
    root_path=root_path,
    s_image_folder=s_image_folder,
    u_image_folder=u_image_folder,
    training=True,
    copy_data=copy_data,
  )

  #iter_supervised = None
  ipe = len(unsupervised_loader)

  logger.info(f"iterations per epoch: {ipe}")

  # -- init optimizer and scheduler
  #scaler = torch.cuda.amp.GradScaler(enabled=use_fp16)
  
  # To Do: Optimizer Doble check learning rate
  optimizer = torch.optim.Adam(encoder.parameters(), 0.0001, weight_decay=5e-4)

  def train_loop_fn(supervised_loader, unsupervised_loader, epoch):
      tracker = xm.RateTracker()
      # -- TRAINING LOOP
      best_loss = None

      unsupervised_sampler.set_epoch(epoch)

      loss_meter = AverageMeter()
      ploss_meter = AverageMeter()
      rloss_meter = AverageMeter()
      time_meter = AverageMeter()
      data_meter = AverageMeter()

      for itr, udata in enumerate(unsupervised_loader):
          def load_imgs(supervised_loader):
              # -- unsupervised imgs
              uimgs = [u.to(device, non_blocking=True) for u in udata[:-1]]
              # -- supervised imgs
              global iter_supervised
              try:
                  sdata = next(iter_supervised)
              except Exception:
                  iter_supervised = iter(supervised_loader)
                  sdata = next(iter_supervised)
              finally:
                  idx = sdata[1].clone().detach()
                  idx = idx.to(device)

                  labels_matrix = torch.zeros(len(idx), idx.max()+1, device = device).scatter_(1, idx.unsqueeze(1), 1.)
  
                  labels_matrix = labels_matrix.to(device)
                  labels = torch.cat([labels_matrix for _ in range(supervised_views)])
                  simgs = [s.to(device, non_blocking=True) for s in sdata[:-1]]
              # -- concatenate supervised imgs and unsupervised imgs
              imgs = simgs + uimgs
              return imgs, labels
          
          imgs, labels = load_imgs(supervised_loader)
          
          def train_step():

              #with torch.cuda.amp.autocast(enabled=use_fp16):
              optimizer.zero_grad()

              # --
              # h: representations of 'imgs' before head
              # z: representations of 'imgs' after head
              # -- If use_pred_head=False, then encoder.pred (prediction
              #    head) is None, and _forward_head just returns the
              #    identity, z=h
              h, z = encoder(imgs, return_before_head=True)

              # Compute paws loss in full precision
              #with torch.cuda.amp.autocast(enabled=False):

              # Step 1. convert representations to fp32
              h, z = h.float(), z.float()

              # Step 2. determine anchor views/supports and their
              #         corresponding target views/supports
              # --
              num_support = (
                  supervised_views * s_batch_size * classes_per_batch
              )

              # --
              anchor_supports = z[:num_support]
              anchor_views = z[num_support:]
              # --
              target_supports = h[:num_support].detach()
              target_views = h[num_support:].detach()
              target_views = torch.cat(
                  [
                      target_views[u_batch_size : 2 * u_batch_size],
                      target_views[:u_batch_size],
                  ],
                  dim=0,
              )

              # Step 3. compute paws loss with me-max regularization
              ploss, me_max = my_loss_func(
                  anchor_views=anchor_views,
                  anchor_supports=anchor_supports,
                  anchor_support_labels=labels,
                  target_views=target_views,
                  target_supports=target_supports,
                  target_support_labels=labels,
              )

              loss = ploss + me_max
              loss.backward()

              xm.optimizer_step(optimizer)

              if itr % FLAGS.get("log_steps") == 0:
                  print(
                      "[xla:{}]({}) Loss={:.5f} Rate={:.2f} GlobalRate={:.2f} Time={}".format(
                          xm.get_ordinal(),
                          itr,
                          loss.item(),
                          tracker.rate(),
                          tracker.global_rate(),
                          time.asctime(),
                      ),
                      flush=True,
                  )
 
              return (float(loss), float(ploss), float(me_max))

          (loss, ploss, rloss) = train_step()
          loss_meter.update(loss)
          ploss_meter.update(ploss)
          rloss_meter.update(rloss)

  data, pred, target = None, None, None

  start_epoch = 0
  end_epoch = FLAGS.get("num_epochs")

  train_supervised_loader = pl.MpDeviceLoader(supervised_loader, device)
  train_unsupervised_loader = pl.MpDeviceLoader(unsupervised_loader, device)

  for epoch in range(start_epoch, end_epoch):
      train_loop_fn(train_supervised_loader, train_unsupervised_loader, epoch)
      xm.master_print("Finished training epoch {}".format(epoch))
  #return accuracy, data, pred, target
  return 0, 0, 0, 0

In [55]:
import time

In [ ]:
# Start training processes
def _mp_fn(rank, flags):
  global FLAGS
  FLAGS = flags
  torch.set_default_tensor_type('torch.FloatTensor')
  accuracy, data, pred, target = train_resnet18()

xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=FLAGS['num_cores'], start_method='fork')

[xla:0](0) Loss=0.00342 Rate=0.00 GlobalRate=0.00 Time=Sat Aug 21 18:20:00 2021
[xla:6](0) Loss=0.29423 Rate=0.00 GlobalRate=0.00 Time=Sat Aug 21 18:20:10 2021
[xla:2](0) Loss=0.04341 Rate=0.00 GlobalRate=0.00 Time=Sat Aug 21 18:20:11 2021
[xla:3](0) Loss=-0.01230 Rate=0.00 GlobalRate=0.00 Time=Sat Aug 21 18:20:18 2021
[xla:5](0) Loss=0.00045 Rate=0.00 GlobalRate=0.00 Time=Sat Aug 21 18:20:19 2021
[xla:7](0) Loss=0.19677 Rate=0.00 GlobalRate=0.00 Time=Sat Aug 21 18:20:19 2021
[xla:1](0) Loss=0.02202 Rate=0.00 GlobalRate=0.00 Time=Sat Aug 21 18:20:19 2021
[xla:4](0) Loss=0.00098 Rate=0.00 GlobalRate=0.00 Time=Sat Aug 21 18:20:25 2021
Finished training epoch 0
[xla:7](0) Loss=0.04498 Rate=0.00 GlobalRate=0.00 Time=Sat Aug 21 18:21:01 2021
[xla:5](0) Loss=0.00952 Rate=0.00 GlobalRate=0.00 Time=Sat Aug 21 18:21:02 2021
[xla:4](0) Loss=-0.00206 Rate=0.00 GlobalRate=0.00 Time=Sat Aug 21 18:21:04 2021
[xla:3](0) Loss=0.00430 Rate=0.00 GlobalRate=0.00 Time=Sat Aug 21 18:21:05 2021
[xla:2](0) L